In [25]:
!pip install efficientnet_pytorch

In [1]:
import torch
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from torch import nn, optim
from pytorch_lightning import LightningModule
import torchmetrics
from pytorch_lightning import Trainer
from torch.utils.data import random_split

In [2]:
# Load data
transform = transforms.Compose([transforms.Resize([224, 224]),
                                 transforms.ToTensor()])
dataset = datasets.ImageFolder('../data/raw/garbage/', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

trainset_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [3]:
class EfficientNetModel(LightningModule):
    def __init__(self, num_classes, model_num=0):
        super().__init__()
        assert(model_num >= 0 and model_num <= 7)
        
        self.num_classes = num_classes
        
        # Load pre-trained model
        self.ef = EfficientNet.from_pretrained('efficientnet-b{}'.format(model_num))
        features_size = [1280, 1280, 1408, 1536, 1792, 2048, 2304, 2560]
        
        # Override last layer for garbage classification
        self.ef._fc = nn.Sequential(nn.Linear(in_features=features_size[model_num], out_features=self.num_classes), nn.LogSoftmax(dim=-1))
        
        # Freeze weights of the model except for the last layer
        for name, param in self.ef.named_parameters():
            if name.split(".")[-1] not in ["weight", "bias"]:
                param.requires_grad = False

        # The last layer has different name than the previous layers
        for name, param in self.ef._fc.named_parameters():
            if name.split(".")[-1] in ["weight", "bias"]:
                param.requires_grad = True
        
         # Define the data augmentation transforms
        self.data_augumentation = transforms.Compose([
            transforms.RandomRotation(degrees=90),
            transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(1, 1)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ])
        
        self.criterion = nn.NLLLoss()

    def foreward(self, x):
        out = self.ef(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self.data_augumentation(x)

        logits = self.foreward(x)
        loss = self.criterion(logits, torch.flatten(y))

        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        acc = accuracy(preds, torch.flatten(y))

        self.log("train_loss", loss, on_step=True, on_epoch=True, logger=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.foreward(x)
        loss = self.criterion(logits, torch.flatten(y))

        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        acc = accuracy(preds, torch.flatten(y))

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.foreward(x)
        loss = self.criterion(logits, torch.flatten(y))

        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=self.num_classes)
        acc = accuracy(preds, torch.flatten(y))

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters())


In [4]:
# Train
print("Training...")
model = EfficientNetModel(6, model_num=0)
trainer = Trainer()
trainer.fit(model, trainset_loader, testset_loader)
print("Done")

GPU available: False, used: False


Training...
Loaded pretrained weights for efficientnet-b0


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/carlo/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name      | Type         | Params
-------------------------------------------
0 | ef        | EfficientNet | 4.0 M 
1 | criterion | NLLLoss      | 0     
-------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.061    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/carlo/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/carlo/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/carlo/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which 

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Done


/Users/carlo/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [16]:
trainer = Trainer()
trainer.test(model, testset_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]